In [7]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf
from tensorflow import keras

2025-04-09 00:34:24.700852: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [86]:
train_df = pd.read_csv('train.csv')

In [19]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [20]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [87]:
y_train = np.array(train_df['SalePrice'])

In [25]:
y_train.shape

(1460,)

In [26]:
y_train[0]

208500

In [88]:
train_df = train_df.drop(labels=['Id', 'SalePrice'], axis=1)

In [89]:
num_cols = train_df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = train_df.select_dtypes(include=['object']).columns

In [90]:
train_df[num_cols] = train_df[num_cols].fillna(train_df[num_cols].mean())
train_df[cat_cols] = train_df[cat_cols].fillna("None")

In [91]:
from sklearn.preprocessing import OneHotEncoder

In [92]:
train_df = pd.get_dummies(train_df, columns=cat_cols, drop_first=True)

In [93]:
from sklearn.preprocessing import MinMaxScaler

In [94]:
scaler = MinMaxScaler()

In [95]:
train_df[num_cols] = scaler.fit_transform(train_df[num_cols])

In [40]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 260 entries, MSSubClass to SaleCondition_Partial
dtypes: bool(224), float64(36)
memory usage: 730.1 KB


In [96]:
x_train = np.array(train_df).astype(np.float32)

In [42]:
x_train.shape

(1460, 260)

In [97]:
y_train = np.array(y_train).astype(np.float32)

In [45]:
y_train.shape

(1460,)

In [192]:
model = keras.Sequential([
    layers.Dense(260, activation='relu', input_shape=(x_train.shape[1],)),
    layers.Dense(260, activation='relu'),
    layers.Dense(260, activation='relu'),
    layers.Dense(1)  
])

/Users/chuck/python-projects/kaggle/ames_housing_prices/venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [193]:
model.compile(
    optimizer="rmsprop",
    loss='mse',             
    metrics=['mae']         
)

In [194]:
history = model.fit(x_train, y_train, validation_split=0.2, epochs=30, batch_size=32)

Epoch 1/30
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 38500712448.0000 - mae: 179819.2969 - val_loss: 36246646784.0000 - val_mae: 171972.2344
Epoch 2/30
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 33908191232.0000 - mae: 165018.6719 - val_loss: 20614178816.0000 - val_mae: 118821.4219
Epoch 3/30
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 14881936384.0000 - mae: 96277.6250 - val_loss: 6519716352.0000 - val_mae: 50039.4961
Epoch 4/30
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5202660864.0000 - mae: 50412.5117 - val_loss: 5227484672.0000 - val_mae: 47001.1562
Epoch 5/30
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5318816256.0000 - mae: 48202.6523 - val_loss: 4533513728.0000 - val_mae: 46890.9023
Epoch 6/30
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3743261696.0000 - mae: 42512.9219 - val_loss: 4024060416.0000 - val_mae: 37884.7578
Epoch 7/30
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2899977472.0000 - mae: 35449.3164 - val_loss: 3501081856.0000 - val_mae: 35651.7

In [191]:
print(y_train.max())
print(y_train.min())

755000.0
34900.0


In [195]:
test_df = pd.read_csv('test.csv')

In [197]:
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [198]:
test_df = test_df.drop(labels=['Id'], axis=1)

In [199]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1455 non-null   object 
 2   LotFrontage    1232 non-null   float64
 3   LotArea        1459 non-null   int64  
 4   Street         1459 non-null   object 
 5   Alley          107 non-null    object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1457 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearBuil

In [200]:
test_num_cols = test_df.select_dtypes(include=['int64', 'float64']).columns
test_cat_cols = test_df.select_dtypes(include=['object']).columns

In [201]:
test_df[test_num_cols] = test_df[test_num_cols].fillna(test_df[test_num_cols].mean())
test_df[test_cat_cols] = test_df[test_cat_cols].fillna("None")

In [202]:
test_df = pd.get_dummies(test_df, columns=test_cat_cols, drop_first=True)

In [203]:
test_df[test_num_cols] = scaler.fit_transform(test_df[test_num_cols])

In [206]:
test_df = test_df.reindex(columns=train_df.columns, fill_value=0)

In [207]:
x_test = np.array(test_df).astype(np.float32)

In [208]:
y_test = model.predict(x_test)

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [209]:
y_test.shape

(1459, 1)

In [210]:
y_test

array([[107315.86],
       [148468.92],
       [177192.78],
       ...,
       [169220.66],
       [106951.36],
       [206699.44]], dtype=float32)

In [211]:
y_test.reshape(-1)

array([107315.86, 148468.92, 177192.78, ..., 169220.66, 106951.36,
       206699.44], dtype=float32)

In [213]:
submission = pd.DataFrame({
    'Id': pd.read_csv('test.csv')['Id'],
    'SalePrice': y_test.flatten()
})
submission.to_csv('submission.csv', index=False)